In [ ]:
%env LLM_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1
%env LLM_API_KEY=替换为自己的Qwen API Key，如果不需要评估，则不需要

In [2]:
import os
import pandas as pd
from openai import OpenAI

# 加载数据

In [3]:
expr_version = 'product_v03_flowise_rrf'

preprocess_output_dir = os.path.join(os.path.pardir, 'outputs', 'v1_20240713')
expr_dir = os.path.join(os.path.pardir, 'experiments', expr_version)

In [4]:
qa_df = pd.read_excel(os.path.join(preprocess_output_dir, 'question_answer.xlsx'))
prediction_df = qa_df[qa_df['dataset'] == 'test'][['uuid', 'question', 'qa_type', 'answer']].rename(columns={'answer': 'ref_answer'})

# 调用Flowise API

In [5]:
import requests

API_URL = "http://192.168.31.92:3000/api/v1/prediction/f2740f69-4244-4438-ad68-752811a3fc7e"

def rag(question):
    payload = {
        "question": question
    }
    response = requests.post(API_URL, json=payload)
    return response.json()

In [6]:
rag('这份报告的发布时间是什么时候')

{'text': '报告日期：2023年12月12日',
 'question': '这份报告的发布时间是什么时候',
 'chatId': 'ed6b5627-3556-4522-bd75-c4c3607f40bf',
 'chatMessageId': 'c11a3d85-06a6-4ee6-a070-4a639914b74a',
 'isStreamValid': False,
 'sessionId': 'ed6b5627-3556-4522-bd75-c4c3607f40bf'}

# 批量预测

In [7]:
from tqdm.auto import tqdm

answer_dict = {}
for idx, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
    uuid = row['uuid']
    question = row['question']
    raw_resp = rag(question)

    answer_dict[question] = {
        'uuid': uuid,
        'ref_answer': row['ref_answer'],
        'gen_answer': raw_resp['text']
    }

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
prediction_df.loc[:, 'gen_answer'] = prediction_df['question'].apply(lambda q: answer_dict[q]['gen_answer'])

In [9]:
prediction_df.sample(5)

,uuid,question,qa_type,ref_answer,gen_answer
385,ddfb44a5-ffa5-4973-a79b-662a0bfeb491,浮动利率房贷在新增房贷中的占比有何变化，为什么会有这种变化？,large_context,浮动利率房贷在新增房贷中的占比上升，这是因为它们通常比固定利率低100个基点以上，这能帮助债...,浮动利率房贷在新增房贷中的占比上升。因为30年房贷利率预计将在2025年一季度下降到6.4%...
320,ecc12483-d88f-41af-93a6-16c48d4ef147,2023年二季度，银行业的商业房地产贷款拖欠率是多少？,detailed,0.8%,2023年二季度，银行业的商业房地产贷款拖欠率为0.8%。
60,774ab320-aba8-45a7-a5f4-791782b46d08,哪些因素影响了欧元区和英国的经济增长？,detailed,就业市场韧性消退、内外部需求回落、通胀和利率水平高企、地缘冲突余波未散,欧洲央行收紧货币政策、通胀高企抑制消费能力、疫情放开的促进效应逐步消减以及全球经济放缓导致出...
174,05696bdf-74d8-4788-8e18-86439fa0bf0c,英格兰银行在6月22日的利率变动了多少基点?,detailed,50基点,英格兰银行在6月22日的利率变动了50基点。
24,1b782370-c7f8-48a8-9647-591bb91f27f7,全球供应链压力指数降至有记录以来最低值的时间点是什么时候？,detailed,2023年10月底,全球供应链压力指数降至有记录以来最低值的时间点是截至2023年10月底。


# 评估

In [10]:
from langchain_openai import ChatOpenAI

judge_llm = ChatOpenAI(
    api_key=os.environ['LLM_API_KEY'],
    base_url=os.environ['LLM_BASE_URL'],
    model_name='qwen2-72b-instruct',
    temperature=0
)

import time

def evaluate(prediction_df):
    """
    对预测结果进行打分
    :param prediction_df: 预测结果，需要包含问题，参考答案，生成的答案，列名分别为question, ref_answer, gen_answer
    :return 打分模型原始返回结果
    """
    prompt_tmpl = """
你是一个经济学博士，现在我有一系列问题，有一个助手已经对这些问题进行了回答，你需要参照参考答案，评价这个助手的回答是否正确，仅回复“是”或“否”即可，不要带其他描述性内容或无关信息。
问题：
<question>
{{question}}
</question>

参考答案：
<ref_answer>
{{ref_answer}}
</ref_answer>

助手回答：
<gen_answer>
{{gen_answer}}
</gen_answer>
请评价：
    """
    results = []

    for _, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        question = row['question']
        ref_answer = row['ref_answer']
        gen_answer = row['gen_answer']

        prompt = prompt_tmpl.replace('{{question}}', question).replace('{{ref_answer}}', str(ref_answer)).replace('{{gen_answer}}', gen_answer).strip()
        result = judge_llm.invoke(prompt).content
        results.append(result)

        time.sleep(1)
    return results

In [11]:
prediction_df['raw_score'] = evaluate(prediction_df)

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
prediction_df['raw_score'].unique()

array(['是', '否'], dtype=object)

In [13]:
prediction_df['score'] = (prediction_df['raw_score'] == '是').astype(int)

In [14]:
prediction_df['score'].mean()

0.78